In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.special import binom
import matplotlib.pyplot as plt
import matplotlib as mpl

# Data

In [ ]:
POSITION_DICT = {0: 'P', 1: 'C', 2: '1B', 3: '2B', 4: '3B', 5: 'SS', 6: 'LF', 7: 'CF', 8: 'RF', 9: 'DH'}

Need to subtract 1 from `position_main` to be able to index positional array's

In [15]:
data = pd.read_csv('../data/processed/test_data.csv').reset_index(drop=True)
# data.rename(columns={'park_clean':'park'}, inplace=True)


data['eliteStatus'] = stats.bernoulli.rvs(p=0.1, size=len(data))
data['eliteStatusPrev'] = data.groupby('playerID')['eliteStatus'].shift(1)
data['position_main'] = data['position_main'] - 1 

data.drop(columns=['birthYear', 'debut'], inplace=True)

# data should be sorted --- Verify
# data.sort_values(['playerID', 'yearID'], inplace=True)

# pos_map = {'DH': 0, 'C': 1, '1B': 2, '2B': 3, '3B': 4, 'SS': 5, 'LF': 6, 'CF': 7, 'RF': 8}
# data['pos'] = data['pos_main'].map(pos_map)

In [16]:
POSITION_DICT = {0: 'C', 1: '1B', 2: '2B', 3: '3B', 4: 'SS', 5: 'LF', 6: 'CF', 7: 'RF', 8: 'DH'}

In [17]:
sorted(data['position_main'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [18]:
# parks = sorted(data.park.unique())
# park_idx = parks.index('AT&T Park')

In [19]:
data.sort_values(['playerID', 'yearID'])

,playerID,yearID,team,AB,HR,position_main,park,playerAge,yearsInMLB,playerFullName,eliteStatus,eliteStatusPrev
0,casteni01,2013,DET,18,0,5,Comerica Park,21,1,Nick Castellanos,0,NaN
1,casteni01,2014,DET,533,11,3,Comerica Park,22,2,Nick Castellanos,1,0.0
2,casteni01,2015,DET,549,15,3,Comerica Park,23,3,Nick Castellanos,0,1.0
3,casteni01,2016,DET,411,18,3,Comerica Park,24,4,Nick Castellanos,0,0.0
4,casteni01,2017,DET,614,26,3,Comerica Park,25,5,Nick Castellanos,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,yelicch01,2014,MIA,582,9,5,Marlins Park,23,2,Christian Yelich,0,0.0
105,yelicch01,2015,MIA,476,7,5,Marlins Park,24,3,Christian Yelich,0,0.0
106,yelicch01,2016,MIA,578,21,5,Marlins Park,25,4,Christian Yelich,0,0.0
107,yelicch01,2017,MIA,602,18,6,Marlins Park,26,5,Christian Yelich,0,0.0


In [5]:
data.groupby(['playerID', 'yearID'])['playerAge'].count().sort_values(ascending=False)

playerID   yearID
yelicch01  2018      1
davisra01  2017      1
machama01  2012      1
hamiljo03  2015      1
           2014      1
                    ..
schumsk01  2005      1
sanchfr01  2011      1
           2010      1
           2009      1
avilaal01  2009      1
Name: playerAge, Length: 222, dtype: int64

# Functions

In [1]:
def theta(alpha, beta, age_trajectory_val):
    inner = alpha + beta + age_trajectory_val
    if inner > 100:
        inner = 100
    return 1 / (1 + np.exp(-inner))

In [7]:
def logit_theta(theta):
    return np.log(theta/(1-theta))

In [8]:
def age_trajectory(age, gamma):
    '''Need to update this be a spline'''
    return gamma[0] + age * gamma[1]

In [9]:
def log_prior(alpha, beta, gamma, nu, omega, tau):
    psum = 0
    
    for x in beta:
        psum += stats.norm(0, tau).logpdf(x)
    
    for x in gamma:
        psum += stats.norm(0, tau).logpdf(x).sum()
    
    for x1, x2 in alpha:
        psum += stats.norm(0, tau).logpdf(x1) + stats.norm(0, tau).logpdf(x2)
    
    # I'm gonna have to mess with this one
    for pos_vals, pos_params in zip(nu, omega):
        for vals, params in zip(pos_vals, pos_params): 
            psum += stats.dirichlet.logpdf(vals, params)
    
    return psum

In [10]:
def log_elite_transitions(pos, prev, curr, nu_params):
    if np.isnan(prev):
        prev = 0
    return np.log(nu_params[pos, int(prev), int(curr)])

In [23]:
def log_posterior(alpha, beta, gamma, nu, omega, tau, data):
    
    """
    Params
    ------
        alpha: Intercept coefficients. Dict w/ pos keys and 2x1 np.array
        beta: Park coefficients. Dict w/ park name keys and 1x1 np.array
        gamma: Spline coefficients. Dict w/ pos keys and 4x1 np.array
        nu: Transition probabilities. Dict w/ pos keys and 2x2 np.array with trans probs
        omega: Dirichlet parameters. 2x2 np.array
        X: Player Data
        
    Returns
    -------
        Log of the Posterior (float)
    """
    
    post_sum = 0
    
    parks = sorted(data.park.unique())
    
    for index, row in data.iterrows():
        # 2nd equation on RHS equation (8) return the LOG of p(θ)
        age = row.playerAge
        pos = row.pos
        park = row.park
        hrs = row.HR
        ab = row.AB
        es_prev = row.eliteStatusPrev
        es_curr = row.eliteStatus
        
        age_traj = age_trajectory(age, gamma[pos])
        
        park_idx = parks.index(park)
        
        rate = theta(alpha[pos, es_curr], beta[park_idx], age_traj)
        log_theta = np.log(rate)
        
        # breakpoint()
        # capture rates > 1 and give them .99 for now
        rate_trunc = rate if rate < 1 else 0.99 
        
        # 1st equation on RHS of equation(8)
        post_sum += stats.binom(n=ab, p=rate_trunc).logpmf(k=hrs)
        # 2nd equation on RHS of equation(8)
        post_sum += log_theta
        # 3rd equation on RHS of equation(8)
        post_sum += log_elite_transitions(pos, es_prev, es_curr, nu)
        
    # 4th equation on the RHS of equation(8)
    post_sum += log_prior(alpha, beta, gamma, nu, omega, tau)
    # print(f'Posterior sum: {post_sum}')
    return post_sum

In [24]:
def log_posterior_alpha_update(alpha, beta, gamma, nu, omega, tau, X):
    
    """
    Params
    ------
        
    Returns
    -------

    """
    
    post_sum = 0
    
    parks = sorted(data.park.unique())
    
    for index, row in X.iterrows():
        # 2nd equation on RHS equation (8) return the LOG of p(θ)
        age = row.playerAge
        pos = row.pos
        park = row.park
        hrs = row.HR
        ab = row.AB
        es_prev = row.eliteStatusPrev
        es_curr = row.eliteStatus
        
        age_traj = age_trajectory(age, gamma[pos])
        
        park_idx = parks.index(park)
        
        rate = theta(alpha[pos, es_curr], beta[park_idx], age_traj)
        log_theta = np.log(rate)
        
        # breakpoint()
        # capture rates > 1 and give them .99 for now
        rate_trunc = rate if rate < 1 else 0.99 
        
        # 1st equation on RHS of equation(8)
        post_sum += stats.binom(n=ab, p=rate_trunc).logpmf(k=hrs)
        # 2nd equation on RHS of equation(8)
        post_sum += log_theta
        
    # 4th equation on the RHS of equation(8)
    alpha_prior = 0
    for x1, x2 in alpha:
        alpha_prior += stats.norm(0, tau).logpdf(x1) + stats.norm(0, tau).logpdf(x2)
    
    post_sum += alpha_prior
    # print(f'Posterior sum: {post_sum}')
    return post_sum

In [25]:
def log_posterior_beta_update(alpha, beta, gamma, nu, omega, tau, X):
    
    """
    Params
    ------
        
    Returns
    -------

    """
    
    post_sum = 0
    
    parks = sorted(data.park.unique())
    
    for index, row in X.iterrows():
        # 2nd equation on RHS equation (8) return the LOG of p(θ)
        age = row.playerAge
        pos = row.pos
        park = row.park
        hrs = row.HR
        ab = row.AB
        es_prev = row.eliteStatusPrev
        es_curr = row.eliteStatus
        
        age_traj = age_trajectory(age, gamma[pos])
        
        park_idx = parks.index(park)
        
        rate = theta(alpha[pos, es_curr], beta[park_idx], age_traj)
        log_theta = np.log(rate)
        
        # breakpoint()
        # capture rates > 1 and give them .99 for now
        rate_trunc = rate if rate < 1 else 0.99 
        
        # 1st equation on RHS of equation(8)
        post_sum += stats.binom(n=ab, p=rate_trunc).logpmf(k=hrs)
        # 2nd equation on RHS of equation(8)
        post_sum += log_theta
        
    # 4th equation on the RHS of equation(8)
    beta_prior += stats.norm(0, tau).logpdf(beta).sum()
    
    post_sum += beta_prior
    # print(f'Posterior sum: {post_sum}')
    return post_sum

In [26]:
def log_posterior_gamma_update(alpha, beta, gamma, nu, omega, tau, X):
    
    """
    Params
    ------
        
    Returns
    -------

    """
    
    post_sum = 0
    
    parks = sorted(data.park.unique())
    
    for index, row in X.iterrows():
        # 2nd equation on RHS equation (8) return the LOG of p(θ)
        age = row.playerAge
        pos = row.pos
        park = row.park
        hrs = row.HR
        ab = row.AB
        es_prev = row.eliteStatusPrev
        es_curr = row.eliteStatus
        
        age_traj = age_trajectory(age, gamma[pos])
        
        park_idx = parks.index(park)
        
        rate = theta(alpha[pos, es_curr], beta[park_idx], age_traj)
        log_theta = np.log(rate)
        
        # breakpoint()
        # capture rates > 1 and give them .99 for now
        rate_trunc = rate if rate < 1 else 0.99 
        
        # 1st equation on RHS of equation(8)
        post_sum += stats.binom(n=ab, p=rate_trunc).logpmf(k=hrs)
        # 2nd equation on RHS of equation(8)
        post_sum += log_theta
        
    # 4th equation on the RHS of equation(8)
    gamma_prior = 0
    for x in gamma:
        gamma_prior += stats.norm(0, tau).logpdf(x).sum()
    
    post_sum += gamma_prior
    # print(f'Posterior sum: {post_sum}')
    return post_sum

In [12]:
def transitions(players: pd.core.groupby.generic.DataFrameGroupBy, curr_params):
    for index, player in players:
        elite_status = player[['eliteStatus', 'eliteStatusPrev']].values
        for curr, prev in elite_status:
            curr = int(curr)
            prev = int(prev) if ~np.isnan(prev) else prev
            if np.isnan(prev):
                continue
            curr_params[prev][curr] += 1
    return curr_params

In [13]:
def update_omegas(data: pd.DataFrame, position: int, curr_params: np.array):
    curr_param_set = curr_params[position, :, :]
    pos = data[data.pos == position]
    players = pos.groupby('playerID')
    return transitions(players, curr_param_set)

In [14]:
def update_nus(nu, omega):
    pos, rows, cols = nu.shape
    for i in range(pos):
        for row in range(rows):
            nu[i, row, :] = stats.dirichlet.rvs(alpha=omega[i, row, :])
    return nu

### Forward-summing Backward sampling

In [15]:
def forward_summing(P, player, alpha, beta, gamma):
    """
        Args
        ----
            P (np.array) : position specific elite transition matrix
            years (int) : number of years played at a given position for a player
            alpha (np.array) : position specific intercepts
            beta (float) : park specific coefs
            gamma (np.array) : position specific spline coefs
        
        Returns
        ------
            M: a transition matrix thingy
    """
    # breakpoint()
    
    pos_0 = player.pos[0]
    init_P = P[pos_0, :, :]
    
    years = len(player)
    
    pi = np.array([init_P[0, 1], init_P[1, 0]])
    
    M = np.zeros((years, 2))
    M[0, :] = pi
    
    for yr in range(1, years):
        pos_n = player.pos[yr]
        new_P = P[pos_n, :, :]
        
        Rtemp = M[yr-1, :].dot(new_P).reshape(1, -1)
        
        age = player.playerAge[yr]
        ab = player.AB[yr]
        hrs = player.HR[yr]
        
        age_traj_val = age_trajectory(age, gamma[pos_n])
        theta0 = theta(alpha[pos_n, 0], beta[pos_n], age_traj_val)
        theta1 = theta(alpha[pos_n, 1], beta[pos_n], age_traj_val)
        
        p0 = stats.binom(n=ab, p=theta0).pmf(k=hrs)
        p1 = stats.binom(n=ab, p=theta1).pmf(k=hrs)
        
        # does it make sense to make these values arbitrarily small
        # should something else happen here?
        if p0 == 0:
            p0 = 1e-32
        
        if p1 == 0:
            p1 = 1e-32
        
        Rtemp *= np.array([p0, p1])
        Rtemp /= Rtemp.sum()
        
        M[yr, :] = Rtemp
    
    return M

In [16]:
def back_sampling(P, M, player):
    """
        Args
        ----

        Returns
        -------

    """
    
    pos_0 = player.pos[0]
    init_P = P[pos_0, :, :]
    
    years = len(player)
    
    prob_e_last = M[years-1, 1]
    last_e = stats.bernoulli(p=prob_e_last).rvs()
    
    samples = [last_e]
    for yr in range(years-1, 0, -1):
        pos_n = player.pos[yr]
        new_P = P[pos_n, :, :]
        
        e = last_e
        Rtemp = M[yr, :]
        Rtemp *= np.array([new_P[0, e], new_P[1, e]])
        Rtemp /= sum(Rtemp)
        
        last_e = stats.bernoulli(p=Rtemp[1]).rvs()
        samples.append(last_e)
        
    return np.array(list(reversed(samples)))

# Initialized Params

In [17]:
# tau = 100

# # Park coefficients
# beta_params = stats.norm.rvs(loc=0, scale=tau, size=len(data.park.unique()))
# # Intercept coefficients subject to alpha0 < alpha1
# alpha_params = stats.multivariate_normal.rvs(mean=0, cov=tau, size=(9, 2)).sort()
# # Spline coefficients
# gamma_params = stats.norm.rvs(loc=0, scale=tau, size=36).reshape(9, 4)
# # Transition probabilities for elite status
# nu_params = stats.dirichlet.rvs(alpha=np.array([1, 1]), size=18).reshape(9, 2, 2)
# # Parameters of dirichlet used to get transition probabilities
# omega_params = np.ones((9, 2, 2))

In [ ]:
PARAMS = {
    'tau': 100,
    'proposal_var': 10
}

In [18]:
def gibbs_sampler(samples, data, params):
    
    tau = params['tau']
    prop_var = params['proposal_var']
    proposal_distribution = stats.norm(loc=0, scale=prop_var)
    
    alpha = stats.multivariate_normal.rvs(mean=0, cov=tau, size=(9, 2))
    alpha.sort()
    beta = stats.norm.rvs(loc=0, scale=tau, size=len(data.park.unique()))
    gamma = stats.norm.rvs(loc=0, scale=tau, size=36).reshape(9, 4)
    nu = stats.dirichlet.rvs(alpha=np.array([1, 1]), size=18).reshape(9, 2, 2)
    omega = np.ones((9, 2, 2))
    elite = data.eliteStatus.values
    
    parks = sorted(data.park.unique())
    
    alpha_samples = [alpha]
    beta_samples = [beta]
    gamma_samples = [gamma]
    nu_samples = [nu]
    # elite_samples = [elite]
    
    alpha_accept = 0
    beta_accept = 0
    gamma_accept = 0
    
    # STEPS = 5
    
    for n in range(samples):
        # alpha update
        alpha_prop = alpha + proposal_distribution.rvs(size=(9, 2))
        alpha_prop.sort()
        rho_a = log_posterior_alpha_update(alpha_prop, beta, gamma, nu, omega, tau, data) - log_posterior_alpha_update(alpha, beta, gamma, nu, omega, tau, data)
        if np.isnan(rho_a):
            rho_a = 0
        rho_a = min(0, rho_a)
        accept = np.random.rand()
        if rho_a > np.log(accept):
            alpha = alpha_prop
        alpha_samples.append(alpha)

        # beta update
        beta_prop = beta + proposal_distribution.rvs(size=len(parks))
        rho_b = log_posterior_beta_update(alpha, beta_prop, gamma, nu, omega, tau, data) - log_posterior_beta_update(alpha, beta, gamma, nu, omega, tau, data)
        if np.isnan(rho_b):
            rho_b = 0
        rho_b = min(0, rho_b)
        accept = np.random.rand()
        if rho_b > np.log(accept):
            beta = beta_prop
        beta_samples.append(beta)

        # gamma update
        gamma_prop = gamma + proposal_distribution.rvs(size=36).reshape(9, 4)
        rho_g = log_posterior_gamma_update(alpha, beta, gamma_prop, nu, omega, tau, data) - log_posterior_gamma_update(alpha, beta, gamma, nu, omega, tau, data)
        if np.isnan(rho_g):
            rho_g = 0
        rho_g = min(0, rho_g)
        accept = np.random.rand()
        if rho_g > np.log(accept):
            gamma = gamma_prop
        gamma_samples.append(gamma)

        # omega update
        omega = np.array([update_omegas(data=data, position=i, curr_params=omega) for i in range(9)])
        # nu update
        nu = update_nus(nu, omega)
        nu_samples.append(nu)

        # breakpoint()
        # elite status update
        elite_status_update = []
        for index, player in data.groupby('playerID'):
            player.reset_index(drop=True, inplace=True)
            forward_sum = forward_summing(nu, player, alpha, beta, gamma)
            back_samp = back_sampling(nu, forward_sum, player)
            elite_status_update.extend(back_samp)

        elite = elite_status_update

        if n % 10 == 0:
            print(n)
            
    return (alpha_samples, alpha_accept), (beta_samples, beta_accept), (gamma_samples, gamma_accept), nu_samples, elite

In [27]:
import time

In [28]:
start = time.time()
things = gibbs_sampler(samples=10000, data=data, params=PARAMS)
end = time.time()
print(f'The sampler took {(end - start) / 60} Minutes')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [40]:
len(things[0])
# len(things[1])
# len(things[2])
# len(things[3])
# len(things[4])

3

## Let's do some spline stuff since it doesn't exist

In [21]:
data_c = data.copy()
data_c['hr_rate'] = data_c['HR'] / data_c['AB']

In [22]:
data_c

,playerID,yearID,team,AB,HR,position_main,park,playerAge,yearsInMLB,playerFullName,eliteStatus,eliteStatusPrev,hr_rate
0,casteni01,2013,DET,18,0,5,Comerica Park,21,1,Nick Castellanos,0,NaN,0.000000
1,casteni01,2014,DET,533,11,3,Comerica Park,22,2,Nick Castellanos,1,0.0,0.020638
2,casteni01,2015,DET,549,15,3,Comerica Park,23,3,Nick Castellanos,0,1.0,0.027322
3,casteni01,2016,DET,411,18,3,Comerica Park,24,4,Nick Castellanos,0,0.0,0.043796
4,casteni01,2017,DET,614,26,3,Comerica Park,25,5,Nick Castellanos,0,0.0,0.042345
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,yelicch01,2014,MIA,582,9,5,Marlins Park,23,2,Christian Yelich,0,0.0,0.015464
105,yelicch01,2015,MIA,476,7,5,Marlins Park,24,3,Christian Yelich,0,0.0,0.014706
106,yelicch01,2016,MIA,578,21,5,Marlins Park,25,4,Christian Yelich,0,0.0,0.036332
107,yelicch01,2017,MIA,602,18,6,Marlins Park,26,5,Christian Yelich,0,0.0,0.029900
